## MachineLearningEngine Class

The MachineLearningEngine class is builds on the CoreEngine class. The CoreEngine class serves as a parent class engines that focus on data, while the MachineLearningEngine class is for engines that focus on learning from data.

In [ ]:
from src.StreamPort.ml.MachineLearningEngine import MachineLearningEngine

#Creates an empty MachineLearningEngine object and prints it
engine = MachineLearningEngine()
engine.print()

## MachineLearningAnalysis Class

The MachineLearningAnalysis class is builds on the class Analysis. The Analysis class that is used to perform analysis on the data. 

In [ ]:
from src.StreamPort.ml.MachineLearningAnalysis import MachineLearningAnalysis

#Creates an empty MachineLearningAnalysis obejct and prints it
analysis = MachineLearningAnalysis()
analysis.print()

## Read the CSV File  

This method is used for the reading the data from the csv file. 

In [1]:
from src.StreamPort.ml.MachineLearningEngine import MachineLearningEngine
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np


#Creates an empty MachineLearningEngine object and prints it
path = 'feature_list.csv'
engine = MachineLearningEngine()
engine.read_csv(path)

engine.print()

#print("Create a list of analysis object and prints it" )
#for analysis in engine._analyses:
    #print(f"Analysis: {analysis.name}")
    #for key, value in analysis.data.items():
         #print(f"{key}: {value}")
    #print("\n")

rownames = engine.get_analyses_names()
print("Analysename: ", rownames)

mat = engine.get_data()
mat.index = rownames
print("Matrix: \n", mat)

# Use the matrix mat to perform a Principle Components Analysis (PCA) and plot the scores
# the labels of the matrix should be the cols as x and row names as analyses names

pca = PCA(n_components=2)
scores = pca.fit_transform(mat)

plt.scatter(scores[:, 0], scores[:, 1])

plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.title("PCA Scores")
plt.show()


# Make a ProcessingSettings child with PCA method and create a results object to hold the PCA results
    #MakePCA(ProcessingSettings) the second MakePCASKL(MakePCA)
# Add the ProcessingSettings to the _settings attribute with add settings
# Create a method in the ML engine to perfom PCA and collect the results
# The results are added to the _results atribute of the engine
# make a plot method in the ML engine for the PCA results and classes


Structure of the CSV file: {'number_of_rows': 45, 'number_of_columns': 4445}

MachineLearningEngine 
  name: None 
  author: None 
  path: None 
  date: 2024-06-03 14:07:08.882687 
  analyses: 45 
  settings: 0 

Analysename:  ['M200429099', 'M200429098', 'M200429097', 'M200429096', 'M200429000A', 'M200422149', 'M200422148', 'M200422147', 'M200422146', 'M200422000A', 'M200414008', 'M200414007', 'M200414006', 'M200414005', 'M200414000A', 'M200408028', 'M200408027', 'M200408026', 'M200408025', 'M200408000A', 'M200403008', 'M200403007', 'M200403006', 'M200403005', 'M200403000A', 'M200401071', 'M200401070', 'M200401069', 'M200401068', 'M200401000A', 'M200320004', 'M200320003', 'M200320002', 'M200320001', 'M200320000A', 'M200318004', 'M200318003', 'M200318002', 'M200318001', 'M200318000A', 'M200317010', 'M200317009', 'M200317008', 'M200317007', 'M200317000A']
Matrix: 
              M100_R148_13791  M100_R178_13767  M100_R585_13804  \
M200429099          0.000000         0.000000         0.0